# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/CS_PhD.gml")

In [3]:
print(nx.info(g))

Graph with 1025 nodes and 1043 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 1025 nodes and 1043 edges
Graph with 1025 nodes and 1043 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/CS_PhD-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/CS_PhD-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/CS_PhD-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/CS_PhD-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

1025

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[112,
 15,
 21,
 11,
 18,
 189,
 155,
 32,
 22,
 2,
 14,
 82,
 136,
 83,
 6,
 42,
 78,
 44,
 119,
 98,
 169,
 80,
 86,
 214,
 122,
 62,
 290,
 177,
 150,
 49,
 104,
 105,
 117,
 386,
 229,
 134,
 46,
 102,
 187,
 145,
 320,
 144,
 50,
 251,
 38,
 258,
 9,
 507,
 359,
 12,
 327,
 165,
 433,
 8,
 5,
 206,
 208,
 247,
 719,
 70,
 267,
 243,
 300,
 266,
 149,
 347,
 4,
 107,
 501,
 63,
 424,
 92,
 35,
 73,
 412,
 465,
 113,
 65,
 152,
 127,
 313,
 60,
 388,
 110,
 311,
 306,
 550,
 382,
 41,
 449,
 90,
 130,
 445,
 269,
 702,
 590,
 115,
 437,
 167,
 345,
 354,
 29,
 31,
 67,
 721,
 363,
 0,
 204,
 560,
 470,
 17,
 342,
 239,
 241,
 124,
 56,
 567,
 725,
 438,
 287,
 511,
 402,
 566,
 873,
 160,
 172,
 213,
 148,
 552,
 183,
 93,
 316,
 332,
 477,
 219,
 13,
 376,
 418,
 39,
 447,
 344,
 743,
 891,
 481,
 253,
 443,
 582,
 216,
 599,
 621,
 179,
 406,
 143,
 26,
 28,
 910,
 282,
 522,
 153,
 671,
 203,
 205,
 934,
 197,
 451,
 126,
 407,
 409,
 574,
 493,
 260,
 47,
 277,
 396,
 191,
 904,

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{112: 1025,
 15: 1024,
 21: 1023,
 11: 1022,
 18: 1021,
 189: 1020,
 155: 1019,
 32: 1018,
 22: 1017,
 2: 1016,
 14: 1015,
 82: 1014,
 136: 1013,
 83: 1012,
 6: 1011,
 42: 1010,
 78: 1009,
 44: 1008,
 119: 1007,
 98: 1006,
 169: 1005,
 80: 1004,
 86: 1003,
 214: 1002,
 122: 1001,
 62: 1000,
 290: 999,
 177: 998,
 150: 997,
 49: 996,
 104: 995,
 105: 994,
 117: 993,
 386: 992,
 229: 991,
 134: 990,
 46: 989,
 102: 988,
 187: 987,
 145: 986,
 320: 985,
 144: 984,
 50: 983,
 251: 982,
 38: 981,
 258: 980,
 9: 979,
 507: 978,
 359: 977,
 12: 976,
 327: 975,
 165: 974,
 433: 973,
 8: 972,
 5: 971,
 206: 970,
 208: 969,
 247: 968,
 719: 967,
 70: 966,
 267: 965,
 243: 964,
 300: 963,
 266: 962,
 149: 961,
 347: 960,
 4: 959,
 107: 958,
 501: 957,
 63: 956,
 424: 955,
 92: 954,
 35: 953,
 73: 952,
 412: 951,
 465: 950,
 113: 949,
 65: 948,
 152: 947,
 127: 946,
 313: 945,
 60: 944,
 388: 943,
 110: 942,
 311: 941,
 306: 940,
 550: 939,
 382: 938,
 41: 937,
 449: 936,
 90: 935,
 130: 934,
 445

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/Collab-CSPhD/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/Collab-CSPhD/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True